In [68]:
from openai import OpenAI
from pydantic import BaseModel

In [69]:
class Think(BaseModel):
    thought: str

    def forward(self):
        """Return the thought."""
        return self.thought


class Add(BaseModel):
    values: list[str]

    def forward(self):
        """Return the sum of the values."""
        return sum([float(value) for value in self.values])


class ToolCalls(BaseModel):
    tool_calls: list[Think | Add]


In [72]:
client = OpenAI()

response = client.responses.parse(
    model='gpt-4o-2024-08-06',
    input=[
        {
            'role': 'system',
            'content': 'Answer the question by creating a tool call. I will then execute the tool call and return the result to you.',
        },
        {'role': 'user', 'content': 'What is 1 + 2 + 3?'},
    ],
    text_format=ToolCalls,
)

In [73]:
response.output_parsed

ToolCalls(tool_calls=[Add(values=['1', '2', '3'])])

In [85]:
for call in response.output_parsed.tool_calls:
    result = call.forward()
    print(f'Tool: {call.__class__.__name__}, Result: {result}')
    # create 'arguments' dictionary
    arguments = call.model_dump()
    print(arguments)

Tool: Add, Result: 6.0
{'values': ['1', '2', '3']}
